In [ ]:
import keyring        # for loading api token
import urllib.request # for encoding URL parameters
import pandas as pd   # for handling data frames
import json           # for handling json
import os             # for outputting the absolute path of the file containing the data

# Authentication

Please read the Authentication section of the [README](README.ipynb) to set up your acess token for the following code.  <span style="color:red"> WARNING: the below code will load the access token you saved from the README.ipynb</span>

In [ ]:
token = keyring.get_password("system", "canvas_token");
print("Loaded token with %d characters." % (len(token)))

# Configuration

The following configuration does not frequently change. Input your course ID found in the url of your canvas course.

In [ ]:
course_id = int(input("Enter your course here: "))

This next one can change based on your assignment  
This can be found https://usfca.instructure.com/courses/{course_id}/assignments/{assignment_id}

In [ ]:
assignment_id = int(input("Enter your assignment here: "))

In [ ]:
config = {
        'base_url': 'https://usfca.instructure.com',
        'course_id': course_id,
        'assignment': assignment_id
}

# Request Setup

The following sets up the REST API request to list students:

<https://canvas.instructure.com/doc/api/assignments.html#method.assignments_api.index>

In [ ]:
api_format = '{base_url}/api/v1/courses/{course_id}/assignments/{assignment}'
api_url = api_format.format(**config)

print("URL:", api_url)

In [ ]:
params = {
    'per_page': 1
}

encoded = urllib.parse.urlencode(params)
print("Params:", encoded) # do not output api key 

In [ ]:
rest_call = '{}?access_token={}&{}'.format(api_url, token, encoded)
print("REST call is %d characters." % (len(rest_call)))

# Fetch Data

Fetch the JSON data using pandas.

In [ ]:
# due to the format of this json object, some preprocessing using the json library needs to be done then converted to a dataframe
with urllib.request.urlopen(rest_call) as url:
    data = json.loads(url.read().decode())
assignment = pd.DataFrame.from_dict(data, orient='index')
print('Loaded {} rows and {} columns.'.format(*assignment.shape))
# The column should be the assignment 
# the rows are the information about the assignment

In [ ]:
# output columns (should only be one for the assignment)
print('Columns:', list(assignment.columns.values))

In [ ]:
assignment.head(5)

# Obtain Rubric

Attempting to obtain rubric information if there is one in the assignment

In [ ]:
try:
    settings = assignment.loc['rubric_settings']
except KeyError:
    print("There is no rubric here please input a new assignment or check that your access token is accurate")

In [ ]:
converted = pd.DataFrame.from_dict(settings)

# Create Rubric Dataframe

Creating dataframe out of the rubric and renaming columns 

In [ ]:
new_df = pd.DataFrame(converted.loc[0, 'rubric_settings'], index=[0]) # creating a dataframe of the rubric

In [ ]:
new_df # should only be as many rows as there are rubrics within the assignment

In [ ]:
# there are many similar names with each endpoint of the data these three columns in particular could be more clear
df = new_df.rename({'id':'rubric_id', 'title':'rubric_name', 'points_possible':'rubric_points'}, axis='columns')

In [ ]:
df # check that the columns were renamed

# Optional

Do this if you already saved a rubric to a file and want to add another rubric to it from another assignment

In [ ]:
# rubrics = pd.read_csv('rubrics.csv')
# df = pd.concat([df, rubrics], axis=0, sort=False)

# Output Results

Output the results to CSV.

In [ ]:
path = 'rubrics.csv'

In [ ]:
df.to_csv(path, header = True, index = False)

In [ ]:
os.path.abspath('rubrics.csv') # the absolute path of the result